# SQL Queries: SparkSQL

### Step 1: Start Spark Session

In [1]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

spark = SparkSession.builder \
            .appName("Spark SQL basic example") \
            .master("local[*]") \
	    	.config("spark.some.config.option", "some-value") \
	    	.getOrCreate()

SparkContext not defined


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/02 22:48:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/02 22:48:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Step 2: Read Parquet files as dataframes

In [3]:
parquet_detalhado = spark.read.parquet("../data/data_cleared/dados_detalhados.parquet")
parquet_detalhado.show()
parquet_detalhado.count()

+-------------+---------+----------------------+--------------------+------------+------+--------------------+-------------------+-------------------+---------------------+-------------------+---------------------+
|id_subsistema|id_estado|nom_modalidadeoperacao|   nom_conjuntousina|   nom_usina|id_ons|                 ceg|       din_instante|val_ventoverificado|flg_dadoventoinvalido|val_geracaoestimada|val_geracaoverificada|
+-------------+---------+----------------------+--------------------+------------+------+--------------------+-------------------+-------------------+---------------------+-------------------+---------------------+
|            N|       MA|             Tipo II-C| Conj. Paulino Neves|   Delta 3 I|MAEDT1|EOL.CV.MA.033682-...|2025-03-01 00:00:00|              2.632|                  0.0|              0.761|                  0.0|
|            N|       MA|             Tipo II-C| Conj. Paulino Neves|  Delta 3 II|MAEDT2|EOL.CV.MA.033683-...|2025-03-01 00:00:00|          

55577477

In [4]:
parquet_principal = spark.read.parquet("../data/data_cleared/dados_principais.parquet")
parquet_principal.show()
parquet_principal.count()

+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+
|id_subsistema|nom_subsistema|id_estado|nom_estado|          nom_usina|   id_ons|ceg|       din_instante|val_geracao|val_geracaolimitada|val_disponibilidade|val_geracaoreferencia|val_geracaoreferenciafinal|cod_razaorestricao|cod_origemrestricao|
+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+
|            N|         NORTE|       MA|  MARANHAO|CONJ. PAULINO NEVES|CJU_MAPLN|  -|2022-12-01 00:00:00|    283.132|               NULL|              319.2|               191.83|                      NULL|              NULL|               NULL|
|            N| 

9503712

In [5]:
from pyspark.sql.functions import col, when

parquet_principal = parquet_principal.withColumn(
    "constrained_off",
    when(col("val_disponibilidade") - col("val_geracao") > 0,
         col("val_disponibilidade") - col("val_geracao"))
    .otherwise(0)
)

parquet_principal = parquet_principal.withColumn(
    "percentual_constrained",
    when(col("val_disponibilidade") > 0,
         (col("constrained_off") / col("val_disponibilidade")) * 100)
    .otherwise(0)
)

parquet_principal.show()

+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+-------------------+----------------------+
|id_subsistema|nom_subsistema|id_estado|nom_estado|          nom_usina|   id_ons|ceg|       din_instante|val_geracao|val_geracaolimitada|val_disponibilidade|val_geracaoreferencia|val_geracaoreferenciafinal|cod_razaorestricao|cod_origemrestricao|    constrained_off|percentual_constrained|
+-------------+--------------+---------+----------+-------------------+---------+---+-------------------+-----------+-------------------+-------------------+---------------------+--------------------------+------------------+-------------------+-------------------+----------------------+
|            N|         NORTE|       MA|  MARANHAO|CONJ. PAULINO NEVES|CJU_MAPLN|  -|2022-12-01 00:00:00|    283.132|               N

In [6]:
queries = {
            "anomalia_constrained_off_extremo": """
            -- Detectar usinas com constrained-off extremamente alto
            SELECT 
                nom_usina,
                nom_estado,
                ano,
                mes,
                AVG(percentual_constrained) as percentual_medio,
                MAX(percentual_constrained) as percentual_max,
                COUNT(*) as registros
            FROM wind_data
            WHERE percentual_constrained > 50  -- Mais de 50% de constrained-off
            GROUP BY nom_usina, nom_estado, ano, mes
            HAVING AVG(percentual_constrained) > 70  -- Média acima de 70%
            ORDER BY percentual_medio DESC
            """,
            
            "anomalia_variacao_geracao": """
            -- Detectar variações bruscas na geração
            WITH geracao_stats AS (
                SELECT 
                    nom_usina,
                    nom_estado,
                    ano,
                    mes,
                    AVG(val_geracao) as geracao_media,
                    STDDEV(val_geracao) as geracao_std
                FROM wind_data
                GROUP BY nom_usina, nom_estado, ano, mes
            )
            SELECT 
                w.nom_usina,
                w.nom_estado,
                w.din_instante,
                w.val_geracao,
                gs.geracao_media,
                gs.geracao_std,
                ABS(w.val_geracao - gs.geracao_media) / gs.geracao_std as z_score
            FROM wind_data w
            JOIN geracao_stats gs ON w.nom_usina = gs.nom_usina 
                AND w.ano = gs.ano 
                AND w.mes = gs.mes
            WHERE ABS(w.val_geracao - gs.geracao_media) / gs.geracao_std > 3  -- Z-score > 3
            ORDER BY z_score DESC
            """,
            
            "anomalia_padrao_temporal": """
            -- Detectar padrões temporais anômalos
            SELECT 
                nom_usina,
                nom_estado,
                hora,
                AVG(percentual_constrained) as percentual_medio_hora,
                COUNT(*) as registros_hora
            FROM wind_data
            GROUP BY nom_usina, nom_estado, hora
            HAVING AVG(percentual_constrained) > (
                SELECT AVG(percentual_constrained) * 1.5 
                FROM wind_data 
                WHERE nom_usina = wind_data.nom_usina
            )
            ORDER BY percentual_medio_hora DESC
            """,
            
            "anomalia_correlacao_vento_geracao": """
            -- Correlação entre vento e geração (usando dados de detalhamento)
            SELECT 
                w.nom_usina,
                w.nom_estado,
                w.ano,
                w.mes,
                CORR(w.val_geracao, d.val_ventoverificado) as correlacao_vento_geracao,
                AVG(w.percentual_constrained) as percentual_constrained_medio
            FROM wind_data w
            JOIN wind_detail d ON w.nom_usina = d.nom_usina 
                AND w.din_instante = d.din_instante
            GROUP BY w.nom_usina, w.nom_estado, w.ano, w.mes
            HAVING CORR(w.val_geracao, d.val_ventoverificado) < 0.3  -- Baixa correlação
                AND AVG(w.percentual_constrained) > 20  -- Alto constrained-off
            ORDER BY percentual_constrained_medio DESC
            """,
            
            "anomalia_tendencia_temporal": """
            -- Detectar tendências temporais anômalas
            WITH monthly_trends AS (
                SELECT 
                    nom_usina,
                    nom_estado,
                    ano,
                    mes,
                    AVG(percentual_constrained) as percentual_medio,
                    LAG(AVG(percentual_constrained), 1) OVER (
                        PARTITION BY nom_usina 
                        ORDER BY ano, mes
                    ) as percentual_anterior
                FROM wind_data
                GROUP BY nom_usina, nom_estado, ano, mes
            )
            SELECT 
                nom_usina,
                nom_estado,
                ano,
                mes,
                percentual_medio,
                percentual_anterior,
                (percentual_medio - percentual_anterior) as variacao,
                CASE 
                    WHEN (percentual_medio - percentual_anterior) > 20 THEN 'AUMENTO_BRUSCO'
                    WHEN (percentual_medio - percentual_anterior) < -20 THEN 'DIMINUICAO_BRUSCA'
                    ELSE 'NORMAL'
                END as tipo_anomalia
            FROM monthly_trends
            WHERE percentual_anterior IS NOT NULL
                AND ABS(percentual_medio - percentual_anterior) > 20
            ORDER BY ABS(percentual_medio - percentual_anterior) DESC
            """,
            
            "anomalia_cluster_espacial": """
            -- Detectar clusters espaciais de anomalias
            WITH state_anomalies AS (
                SELECT 
                    nom_estado,
                    ano,
                    mes,
                    AVG(percentual_constrained) as percentual_estado,
                    COUNT(DISTINCT nom_usina) as num_usinas_afetadas
                FROM wind_data
                WHERE percentual_constrained > 30
                GROUP BY nom_estado, ano, mes
            )
            SELECT 
                nom_estado,
                ano,
                mes,
                percentual_estado,
                num_usinas_afetadas,
                CASE 
                    WHEN percentual_estado > 50 AND num_usinas_afetadas > 5 THEN 'CLUSTER_CRITICO'
                    WHEN percentual_estado > 30 AND num_usinas_afetadas > 3 THEN 'CLUSTER_MODERADO'
                    ELSE 'ISOLADO'
                END as tipo_cluster
            FROM state_anomalies
            WHERE percentual_estado > 30
            ORDER BY percentual_estado DESC, num_usinas_afetadas DESC
            """
        }

In [7]:
from pyspark.sql.functions import year, month, hour, to_timestamp

parquet_principal = parquet_principal.withColumn("ano", year("din_instante"))
parquet_principal = parquet_principal.withColumn("mes", month("din_instante"))
parquet_principal = parquet_principal.withColumn("hora", hour("din_instante"))

parquet_principal.createOrReplaceTempView("wind_data")

In [8]:
parquet_detalhado = parquet_detalhado.withColumn("din_instante", to_timestamp("din_instante"))
parquet_detalhado.createOrReplaceTempView("wind_detail")

In [9]:
resultados = {}
for nome, query in queries.items():
    resultados[nome] = spark.sql(query)

In [10]:
resultados["anomalia_constrained_off_extremo"].show(truncate=False)

[Stage 11:==============================================>           (4 + 1) / 5]

+---------------------------+----------+----+---+----------------+--------------+---------+
|nom_usina                  |nom_estado|ano |mes|percentual_medio|percentual_max|registros|
+---------------------------+----------+----+---+----------------+--------------+---------+
|CONJ. MORRO DO CHAPÉU SUL 2|BAHIA     |2022|12 |100.0           |100.0         |1488     |
|CONJ. SERROTE              |CEARA     |2022|9  |100.0           |100.0         |1392     |
|CONJ. MORRO DO CHAPÉU SUL 2|BAHIA     |2022|8  |100.0           |100.0         |1488     |
|CONJ. LICÍNIO DE ALMEIDA   |BAHIA     |2022|12 |100.0           |100.0         |1488     |
|CONJ. PLANALTINA           |BAHIA     |2022|11 |100.0           |100.0         |1440     |
|CONJ. ALVORADA             |BAHIA     |2022|8  |100.0           |100.0         |1488     |
|CONJ. LICÍNIO DE ALMEIDA   |BAHIA     |2022|9  |100.0           |100.0         |1392     |
|CONJ. PLANALTINA           |BAHIA     |2022|8  |100.0           |100.0         

In [ ]:
import logging
from pyspark.sql.functions import col, mean, max as spark_max, min as spark_min, count, sum as spark_sum

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

extreme_constrained = parquet_principal.filter(col("percentual_constrained") > 70)

count_extremos = extreme_constrained.count()
if count_extremos > 0:
    logger.info(f"Encontradas {count_extremos} ocorrências de constrained-off extremo (>70%)")
        
    usina_analysis = extreme_constrained.groupBy("nom_usina", "nom_estado") \
        .agg(
            mean("percentual_constrained").alias("percentual_medio"),
            spark_max("percentual_constrained").alias("percentual_max"),
            count("*").alias("ocorrencias"),
            spark_sum("constrained_off").alias("constrained_off_total"),
            spark_min("din_instante").alias("primeiro_evento"),
            spark_max("din_instante").alias("ultimo_evento")
        )

    top_usinas = usina_analysis.orderBy(col("constrained_off_total").desc()).limit(10).collect()

    logger.info("Top 10 usinas com mais constrained-off extremo:")
    for row in top_usinas:
        logger.info(f"{row['nom_usina']} ({row['nom_estado']}): {row['constrained_off_total']:.2f} MW, {row['ocorrencias']} eventos")

INFO:__main__:Encontradas 2880946 ocorrências de constrained-off extremo (>70%)
INFO:__main__:Top 10 usinas com mais constrained-off extremo:                   
INFO:__main__:CONJ. SANTA VITÓRIA DO PALMAR (RIO GRANDE DO SUL): 7628958.17 MW, 21179 eventos
INFO:__main__:CONJ. LAGOA DOS BARROS (RIO GRANDE DO SUL): 6888151.20 MW, 28326 eventos
INFO:__main__:CONJ. PEDRA BRANCA (BAHIA): 5619875.67 MW, 26503 eventos
INFO:__main__:CONJ. SANTA CLARA (RIO GRANDE DO NORTE): 5487550.83 MW, 35303 eventos
INFO:__main__:CONJ. MORRO DO CHAPÉU SUL 2 (BAHIA): 5426057.63 MW, 17489 eventos
INFO:__main__:CONJ. SERRA DA BABILÔNIA (BAHIA): 5213800.84 MW, 17968 eventos
INFO:__main__:CONJ. CAJU (RIO GRANDE DO NORTE): 4904370.78 MW, 15407 eventos
INFO:__main__:CONJ. CAMPO LARGO 2 (BAHIA): 4752770.84 MW, 15915 eventos
INFO:__main__:CONJ. LAGOA DOS VENTOS (PIAUI): 4649226.98 MW, 12182 eventos
INFO:__main__:CONJ. CUTIA (RIO GRANDE DO NORTE): 4604494.50 MW, 19789 eventos
